  ALZHEIMER

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout,AveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import tensorflow
import keras
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)

train_dataset = image_generator.flow_from_directory(batch_size=8,
                                                 directory='E:/Downloads/disease detection/alzheimer/Alzheimer_s Dataset/train',
                                                 shuffle=True,
                                                 target_size=(176,208), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=8,
                                                 directory='E:/Downloads/disease detection/alzheimer/Alzheimer_s Dataset/train',
                                                 shuffle=True,
                                                 target_size=(176,208), 
                                                 subset="validation",
                                                 class_mode='categorical')

image_generator_submission = ImageDataGenerator(rescale=1/255) 
submission = image_generator_submission.flow_from_directory(
                                                 directory='E:/Downloads/disease detection/alzheimer/Alzheimer_s Dataset/test',
                                                 shuffle=False,
                                                 target_size=(176,208), 
                                                 class_mode='categorical')

Found 4098 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [3]:
vgg = VGG19(input_shape=(208,176,3), weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)

prediction = Dense(4, activation='softmax')(x)

modelvgg = Model(inputs=vgg.input, outputs=prediction)
modelvgg.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 208, 176, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 208, 176, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 208, 176, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 104, 88, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 104, 88, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 104, 88, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 52, 44, 128)       0     

In [4]:
modelvgg.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=8,
                                            restore_best_weights=True)

In [8]:
modelvgg.fit(train_dataset, epochs=5, validation_data=validation_dataset, callbacks=callback)

Epoch 1/5
513/513 [==============================] - 704s 1s/step - loss: 0.9667 - auc: 0.8551 - val_loss: 1.1580 - val_auc: 0.7696
Epoch 2/5
513/513 [==============================] - 654s 1s/step - loss: 0.6980 - auc: 0.9194 - val_loss: 1.1597 - val_auc: 0.8238
Epoch 3/5
513/513 [==============================] - 658s 1s/step - loss: 0.5357 - auc: 0.9487 - val_loss: 1.0842 - val_auc: 0.8303
Epoch 4/5
513/513 [==============================] - 636s 1s/step - loss: 0.4288 - auc: 0.9662 - val_loss: 0.9865 - val_auc: 0.8526
Epoch 5/5
513/513 [==============================] - 635s 1s/step - loss: 0.3969 - auc: 0.9708 - val_loss: 1.2172 - val_auc: 0.8546


In [10]:
ModelToJSON=modelvgg.to_json()

In [11]:
obj=open("D:/Datasets/AlzheimerDetection.json","w")
obj.write(ModelToJSON)
obj.close()

In [12]:
modelvgg.save_weights("D:/Datasets/AlzheimerDetection.h5")

In [8]:
loss, accuracy = model.evaluate(validation_dataset)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

128/128 [==============================] - 116s 904ms/step - loss: 0.9138 - auc: 0.8983
Loss:  0.9137775897979736
Accuracy:  0.8982507586479187


In [9]:
loss, accuracy = model.evaluate(submission)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

40/40 [==============================] - 155s 4s/step - loss: 0.9301 - auc: 0.9025
Loss:  0.9300808906555176
Accuracy:  0.9025392532348633


In [19]:
test_features=np.array(test_features)
test_target=np.array(test_target)

In [54]:
result = modelvgg.predict(test_features)
pred = np.zeros((1279,1))
for i in range(1279):
    pred[i] = np.max(result[i])
acc = np.sum(np.square((test_target-pred)**2).mean(axis=0))/1279
acc

0.7898894829028174

In [2]:
from tensorflow.keras.models import model_from_json
abc=open("D:/Datasets/AlzheimerDetection.json","r")
loaded_data=abc.read()
model=model_from_json(loaded_data)
model.load_weights("D:/Datasets/AlzheimerDetection.h5")

In [5]:
model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=8,
                                            restore_best_weights=True)

In [6]:
model.fit(train_dataset, epochs=10, validation_data=validation_dataset, callbacks=callback)

Epoch 1/10
513/513 [==============================] - 639s 1s/step - loss: 0.3913 - auc: 0.9730 - val_loss: 1.3240 - val_auc: 0.7979
Epoch 2/10
513/513 [==============================] - 671s 1s/step - loss: 0.2856 - auc: 0.9850 - val_loss: 1.7238 - val_auc: 0.7901
Epoch 3/10
513/513 [==============================] - 649s 1s/step - loss: 0.2627 - auc: 0.9874 - val_loss: 1.0125 - val_auc: 0.8649
Epoch 4/10
513/513 [==============================] - 659s 1s/step - loss: 0.2183 - auc: 0.9916 - val_loss: 1.1666 - val_auc: 0.8476
Epoch 5/10
513/513 [==============================] - 647s 1s/step - loss: 0.1977 - auc: 0.9933 - val_loss: 0.9200 - val_auc: 0.8834
Epoch 6/10
513/513 [==============================] - 647s 1s/step - loss: 0.2028 - auc: 0.9922 - val_loss: 1.0357 - val_auc: 0.8692
Epoch 7/10
513/513 [==============================] - 651s 1s/step - loss: 0.1475 - auc: 0.9966 - val_loss: 0.8719 - val_auc: 0.8901
Epoch 8/10
513/513 [==============================] - 671s 1s/step - 

In [7]:
ModelToJSON=model.to_json()
obj=open("D:/Datasets/AlzheimerDetection.json","w")
obj.write(ModelToJSON)
obj.close()
model.save_weights("D:/Datasets/AlzheimerDetection.h5")

In [ ]:
train_features=[]
train_target=[]
for i in range(4):
    TrainList = os.listdir("E:/Downloads/disease detection/alzheimer/Alzheimer_s Dataset/train/" + str(i))
    for j in TrainList:
        TrainImg = cv2.imread("E:/Downloads/disease detection/alzheimer/Alzheimer_s Dataset/train/" + str(i) + "/" + str(j))
        train_features.append(TrainImg)
        train_target.append(i)

In [ ]:
train_features=np.array(train_features)
train_target=np.array(train_target)
test_features=np.array(test_features)
test_target=np.array(test_target)
train_target=to_categorical(train_target)